In [ ]:
from google.colab import files
uploaded = files.upload()

df_node = pd.read_csv(io.BytesIO(uploaded['annotations.csv']))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
lung_folder = '/content/drive/MyDrive/lung/'

In [ ]:
!ls "/content/drive/My Drive/lung"

kaggle	luna2016  luna_subset  lung_subset


In [ ]:
!pip install pydicom

     |████████████████████████████████| 1.9MB 5.7MB/s 


In [ ]:
!pip install SimpleITK

     |████████████████████████████████| 47.4MB 94kB/s 


In [ ]:
# preprocessing

import pydicom as dicom
import os
import numpy as np
import scipy.ndimage
from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.pyplot as plt


# Load the scans in given folder path
def load_scan(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
    return slices

def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = np.array([scan[0].SliceThickness] + scan[0].PixelSpacing, dtype=np.float32)

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
    
    return image, new_spacing

def plot_3d(image, threshold=-300):
    
    # Position the scan upright, 
    # so the head of the patient would be at the top facing the camera
    p = image.transpose(2,1,0)
    
    verts, faces = measure.marching_cubes(p, threshold)

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], alpha=0.70)
    face_color = [0.45, 0.45, 0.75]
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)

    ax.set_xlim(0, p.shape[0])
    ax.set_ylim(0, p.shape[1])
    ax.set_zlim(0, p.shape[2])

    plt.show()


In [ ]:
# imagenet_utils

import numpy as np
import json

from keras.utils.data_utils import get_file
from keras import backend as K

CLASS_INDEX = None
CLASS_INDEX_PATH = 'https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json'


def preprocess_input(x, dim_ordering='default'):
    if dim_ordering == 'default':
        dim_ordering = K.image_dim_ordering()
    assert dim_ordering in {'tf', 'th'}

    if dim_ordering == 'th':
        x[:, 0, :, :] -= 103.939
        x[:, 1, :, :] -= 116.779
        x[:, 2, :, :] -= 123.68
        # 'RGB'->'BGR'
        x = x[:, ::-1, :, :]
    else:
        x[:, :, :, 0] -= 103.939
        x[:, :, :, 1] -= 116.779
        x[:, :, :, 2] -= 123.68
        # 'RGB'->'BGR'
        x = x[:, :, :, ::-1]
    return x


def decode_predictions(preds, top=5):
    global CLASS_INDEX
    if len(preds.shape) != 2 or preds.shape[1] != 1000:
        raise ValueError('`decode_predictions` expects '
                         'a batch of predictions '
                         '(i.e. a 2D array of shape (samples, 1000)). '
                         'Found array with shape: ' + str(preds.shape))
    if CLASS_INDEX is None:
        fpath = get_file('imagenet_class_index.json',
                         CLASS_INDEX_PATH,
                         cache_subdir='models')
        CLASS_INDEX = json.load(open(fpath))
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        results.append(result)
    return results


In [ ]:
# score

import numpy as np
from skimage import measure


def true_positive(act_blob, pred_blobs, max_dist):
    """checks whether 1 ground truth nodule was predicted correctly"""

    for pred_blob in pred_blobs:
        if np.sqrt(np.abs(np.dot(act_blob - pred_blob, act_blob - pred_blob))) < max_dist:
            return 1
    return 0


def false_positive(act_blobs, pred_blob, max_dist):
    """checks whether 1 predicted nodule is a false positive"""

    for act_blob in act_blobs:
        if np.sqrt(np.abs(np.dot(act_blob - pred_blob, act_blob - pred_blob))) < max_dist:
            return 0
    return 1


def score_slice(act, pred, max_dist):
    """function that takes ground truth mask and predicted mask for one slice and
    outputs the numbers of true positives, false positives, and false negatives.
    max_dist is the maximum distance in pixels allowed between the centroid of ground
    truth nodule and that of predicted nodule in order to consider the predicted
    nodule a true positive.
    """

    act_blobs = map(lambda x: np.array(x.centroid), measure.regionprops(measure.label(act)))
    pred_blobs = map(lambda x: np.array(x.centroid), measure.regionprops(measure.label(pred)))
    true_positives = 0
    false_positives = 0
    for act_blob in act_blobs:
        true_positives += true_positive(act_blob, pred_blobs, max_dist)
    for pred_blob in pred_blobs:
        false_positives += false_positive(act_blobs, pred_blob, max_dist)
    false_negatives = len(act_blobs) - true_positives

    return np.array([true_positives, false_positives, false_negatives])


In [ ]:
# detection

"""Script to create training data for U-net using the LUNA dataset.
   Saves 3 arrays in the luna directory.
   This is a modified version of 
   https://github.com/booz-allen-hamilton/DSB3Tutorial/blob/master/tutorial_code/LUNA_mask_extraction.py
"""

from __future__ import division

from glob import glob

from tqdm import tqdm
import numpy as np
import pandas as pd
import SimpleITK as sitk
from skimage.draw import circle


# luna = 'data/luna2016/'  # make sure the paths work!
file_list = glob('/content/drive/My Drive/lung/luna_subset/subset1/*.mhd')


def get_filename(case):
    for f in file_list:
        if case in f:
            return f


def get_nodules(img_file, biggest=False):
    """Function that returns a list of tuples identifying the nodule locations
       in a file, where each tuple is (x-coord, y-coord, z-coord, diameter). If
       biggest=True, returns only biggest nodule in file.
    """
     
    mini_df = df_node[df_node['file'] == img_file]
    if len(mini_df) == 0:
        return []

    if biggest:
        idx = np.argsort(mini_df['diameter_mm'].values)[-1:]
    else:
        idx = range(len(mini_df))

    x = mini_df['coordX'].values[idx]
    y = mini_df['coordY'].values[idx]
    z = mini_df['coordZ'].values[idx]
    diam = mini_df['diameter_mm'].values[idx]
    return list(zip(x, y, z, diam))


def get_arrays(img_file, biggest=False):
    """Function that returns 3 lists (masks, large_masks, imgs), where each list is
       is a list of numpy arrays. The len of each list is equal to the number of
       nodules in img_file (or 1 if biggest=True); i.e., only one slice is added per
       nodule. The shape of the arrays is 512x512. Two versions of masks are returned:
       masks, which identify nodules by circles of twice the diameter of the ground
       truth; and large_masks, with circles of 4x the ground truth diameter.
    """
    
    nodules = get_nodules(img_file, biggest)
    itk_img = sitk.ReadImage(img_file)
    img = sitk.GetArrayFromImage(itk_img)  # zyx
    origin = np.array(itk_img.GetOrigin())  # xyz
    spacing = np.array(itk_img.GetSpacing())  # xyz
    masks = []
    large_masks = []
    imgs = []
    for nodule in nodules:
        center = np.array(nodule[:3])  # xyz
        v_center = np.rint((center-origin)/spacing).astype(int)  # xyz
        v_diam = int(np.round(nodule[3]/spacing[0]))
        mask = np.zeros((img.shape[1], img.shape[2]), dtype=np.int8)  # yx
        rr, cc = circle(v_center[1], v_center[0], 2*(v_diam/2), shape=mask.shape)
        mask[rr, cc] = 1
        masks.append(mask)
        mask = np.zeros((img.shape[1], img.shape[2]), dtype=np.int8)  # yx
        rr, cc = circle(v_center[1], v_center[0], 4*(v_diam/2), shape=mask.shape)
        mask[rr, cc] = 1
        large_masks.append(mask)
        imgs.append(img[v_center[2], :, :])

    return masks, large_masks, imgs


df_node = pd.read_csv('/content/drive/My Drive/lung/luna2016/annotations.csv')
df_node['file'] = df_node['seriesuid'].apply(get_filename)
df_node = df_node.dropna()

imgs = np.zeros((2000, 512, 512), dtype=np.int16)
masks = np.zeros((2000, 512, 512), dtype=np.int8)
large_masks = np.zeros((2000, 512, 512), dtype=np.int8)
i = 0
for img_file in tqdm(file_list):
    arrays = get_arrays(img_file)
    if len(arrays[0]) == 0:
        continue
    nb = len(arrays[0])
    masks[i:i+nb, :, :] = arrays[0]
    large_masks[i:i+nb, :, :] = arrays[1]
    imgs[i:i+nb, :, :] = arrays[2]
    i += nb

imgs = imgs[:i, :, :]
masks = masks[:i, :, :]
large_masks = large_masks[:i, :, :]

# a few masks have nodule locations outside the 512x512 region for some reason.
# removing these
large_masks_new = np.zeros(large_masks.shape)
masks_new = np.zeros(masks.shape)
imgs_new = np.zeros(imgs.shape)
count = 0
for idx, img in enumerate(imgs):
    if np.sum(masks[idx]) == 0:
        continue
    imgs_new[count] = (img - np.mean(img))/np.std(img)
    masks_new[count] = masks[idx]
    large_masks_new[count] = large_masks[idx]
    count += 1

imgs = imgs_new[:count]
masks = masks_new[:count]
large_masks = large_masks_new[:count]

# shuffle data
np.random.seed(23)
idx = np.random.permutation(imgs.shape[0])
imgs = imgs[idx, :, :]
masks = masks[idx, :, :]
large_masks = large_masks[idx, :, :]

np.save('imgs.npy', imgs)
np.save('masks.npy', masks)
np.save('large_masks.npy', large_masks)


100%|██████████| 5/5 [00:15<00:00,  3.13s/it]


In [ ]:
# unet
from __future__ import division
from __future__ import print_function

import numpy as np
from keras.models import Model, load_model
from keras.layers import Input, merge, concatenate, Convolution2D, MaxPooling2D, UpSampling2D,Conv2D
from keras.optimizers import Adam
from keras import backend as K


luna = '/content/sample_data/'


def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)


def dice_coef_np(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (np.sum(y_true_f) + np.sum(y_pred_f) + 1)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


def get_unet():
    inputs = Input((512, 512, 1))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D((2,2), padding='same')(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D((2,2), padding='same')(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4], axis=3)
    conv6 = Convolution2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Convolution2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=3)
    conv7 = Convolution2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Convolution2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=3)
    conv8 = Convolution2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Convolution2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=3)
    conv9 = Convolution2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Convolution2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Convolution2D(1, 1, 1, activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=conv10)

    model.compile(optimizer=Adam(lr=1.0e-5), loss=dice_coef_loss)

    return model


K.set_image_data_format('channels_last')

imgs = np.load('imgs.npy').astype(np.float32)
masks = np.load('masks.npy').astype(np.int)

masks_new = np.zeros(masks.shape)
imgs_new = np.zeros(imgs.shape)
count = 0
for idx, img in enumerate(imgs):
    if np.sum(masks[idx]) == 0:
        continue
    imgs_new[count] = (img - np.mean(img))/np.std(img)
    masks_new[count] = masks[idx]
    count += 1

imgs = imgs_new[:count]
masks = masks_new[:count]

del imgs_new, masks_new

spl = int(np.round(imgs.shape[0]*0.7))
# tr_imgs = imgs[:spl, np.newaxis, :, :]
# tr_masks = masks[:spl, np.newaxis, :, :]
# ts_imgs = imgs[spl:, np.newaxis, :, :]
# ts_masks = masks[spl:, np.newaxis, :, :]


tr_imgs = imgs[:spl, :, :, np.newaxis]
tr_masks = masks[:spl, :, :, np.newaxis]
ts_imgs = imgs[spl:, :, :, np.newaxis]
ts_masks = masks[spl:, :, :, np.newaxis]

del imgs, masks

unet = get_unet()
#unet = load_model(luna + 'unet9.h5', custom_objects={'dice_coef_loss': dice_coef_loss})
for i in range(20):
    unet.fit(tr_imgs, tr_masks, batch_size=4, epochs=10, verbose=2)
    unet.save(lung_folder + 'unet%d.h5' % i)

    dc = []
    masks_h = unet.predict(tr_imgs, batch_size=4)
    for idx in range(masks_h.shape[0]):
        dc.append(dice_coef_np(tr_masks[idx, 0, :, :], masks_h[idx, 0, :, :]))
    print('train score, epoch:', (i+1)*10, '--', np.mean(dc))

    dc = []
    masks_h = unet.predict(ts_imgs, batch_size=4)
    for idx in range(masks_h.shape[0]):
        dc.append(dice_coef_np(ts_masks[idx, 0, :, :], masks_h[idx, 0, :, :]))
    print('test score, epoch:', (i+1)*10, '--', np.mean(dc))


Epoch 1/10
1/1 - 18s - loss: -5.7707e-03
Epoch 2/10
1/1 - 16s - loss: -5.7725e-03
Epoch 3/10
1/1 - 17s - loss: -5.7745e-03
Epoch 4/10
1/1 - 16s - loss: -5.7766e-03
Epoch 5/10
1/1 - 16s - loss: -5.7787e-03
Epoch 6/10
1/1 - 16s - loss: -5.7808e-03
Epoch 7/10
1/1 - 16s - loss: -5.7830e-03
Epoch 8/10
1/1 - 16s - loss: -5.7852e-03
Epoch 9/10
1/1 - 16s - loss: -5.7874e-03
Epoch 10/10
1/1 - 16s - loss: -5.7896e-03
train score, epoch: 10 -- 0.003910238505105719
test score, epoch: 10 -- 0.003928251154406297
Epoch 1/10
1/1 - 16s - loss: -5.7919e-03
Epoch 2/10
1/1 - 17s - loss: -5.7941e-03
Epoch 3/10
1/1 - 16s - loss: -5.7964e-03
Epoch 4/10
1/1 - 16s - loss: -5.7986e-03
Epoch 5/10
1/1 - 16s - loss: -5.8009e-03
Epoch 6/10
1/1 - 16s - loss: -5.8032e-03
Epoch 7/10
1/1 - 16s - loss: -5.8055e-03
Epoch 8/10
1/1 - 16s - loss: -5.8079e-03
Epoch 9/10
1/1 - 16s - loss: -5.8103e-03
Epoch 10/10
1/1 - 16s - loss: -5.8128e-03
train score, epoch: 20 -- 0.003907387419953345
test score, epoch: 20 -- 0.00392298061

In [ ]:
# segmentation

from __future__ import division

import numpy as np
from skimage import measure
from sklearn.cluster import KMeans
from skimage.transform import resize
from skimage import morphology


def segment_lungs(img, nodule_mask=None):
    mean = np.mean(img)
    std = np.std(img)
    img = (img - mean) / std
    img = img.astype(np.float64)
    middle = img[100:400, 100:400]
    mean = np.mean(middle)
    img_max = np.max(img)
    img_min = np.min(img)

    img[img == img_max] = mean
    img[img == img_min] = mean

    kmeans = KMeans(n_clusters=2).fit(np.reshape(middle, [np.prod(middle.shape), 1]))
    centers = sorted(kmeans.cluster_centers_.flatten())
    threshold = np.mean(centers)
    thresh_img = np.where(img < threshold, 1.0, 0.0)

    eroded = morphology.erosion(thresh_img, np.ones([4, 4]))
    dilation = morphology.dilation(eroded, np.ones([10, 10]))

    labels = measure.label(dilation)
    regions = measure.regionprops(labels)
    good_labels = []
    for prop in regions:
        B = prop.bbox
        if B[2] - B[0] < 475 and B[3] - B[1] < 475 and B[0] > 40 and B[2] < 472:
            good_labels.append(prop.label)
    mask = np.ndarray([512, 512], dtype=np.int8)
    mask[:] = 0

    for N in good_labels:
        mask += np.where(labels == N, 1, 0)

    mask = morphology.dilation(mask, np.ones([10, 10]))

    img = mask * img

    new_mean = np.mean(img[mask > 0])
    new_std = np.std(img[mask > 0])

    old_min = np.min(img)
    img[img == old_min] = new_mean - 1.2 * new_std
    img = (img - new_mean) / new_std

    labels = measure.label(mask)
    regions = measure.regionprops(labels)

    min_row = 512
    max_row = 0
    min_col = 512
    max_col = 0
    for prop in regions:
        B = prop.bbox
        if min_row > B[0]:
            min_row = B[0]
        if min_col > B[1]:
            min_col = B[1]
        if max_row < B[2]:
            max_row = B[2]
        if max_col < B[3]:
            max_col = B[3]
    width = max_col - min_col
    height = max_row - min_row
    if width > height:
        max_row = min_row + width
    else:
        max_col = min_col + height

    img = img[min_row:max_row, min_col:max_col]
    if max_row - min_row < 5 or max_col - min_col < 5:
        return ()
    else:
        mean = np.mean(img)
        img -= mean
        img_min = np.min(img)
        img_max = np.max(img)
        img /= (img_max - img_min)
        new_img = resize(img, [512, 512])
        if isinstance(nodule_mask, np.ndarray):
            nodule_mask = nodule_mask.astype(np.float64)
            new_nodule_mask = resize(nodule_mask[min_row:max_row, min_col:max_col], [512, 512])
        else:
            new_nodule_mask = 0

        return new_img, new_nodule_mask


In [ ]:
# cropping

from __future__ import division
from __future__ import print_function
import pickle
from glob import glob
import numpy as np
import pandas as pd
import pydicom as dicom
from keras import backend as K
from skimage.feature import hog
from skimage.feature import local_binary_pattern
# from segmentation import segment_lungs
from skimage import measure
# from score import false_positive
from sklearn.cluster import k_means
from sklearn.svm import SVC
from keras.models import load_model
from scipy.ndimage.measurements import label
import cv2
import io

import time

# databowl = '/media/data/kaggle/'
# kaggle_datafolder = '/media/data/kaggle/'
# kaggle_metadata = './data/kaggle/'
K.set_image_data_format('channels_last')


def get_patch_coord(centroid, patch_size):
    if centroid[0] < patch_size / 2:
        r_min = 0
        r_max = patch_size
    elif centroid[0] > 512 - patch_size / 2:
        r_min = 512 - patch_size
        r_max = 512
    else:
        r_min = centroid[0] - patch_size / 2
        r_max = centroid[0] + patch_size / 2

    if centroid[1] < patch_size / 2:
        c_min = 0
        c_max = patch_size
    elif centroid[1] > 512 - patch_size / 2:
        c_min = 512 - patch_size
        c_max = 512
    else:
        c_min = centroid[1] - patch_size / 2
        c_max = centroid[1] + patch_size / 2

    return int(r_min), int(r_max), int(c_min), int(c_max)


def read_imgs(patient):
    img_files = glob(lung_folder + 'lung_subset_test/' + patient + '/*')
    slices = [dicom.read_file(img_file) for img_file in img_files]
    slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))
    imgs = np.stack([s.pixel_array for s in slices]).astype(np.float64)
    new_imgs = np.zeros(imgs.shape, dtype=np.float32)
    count = 0
    for img in imgs[int(0.12*imgs.shape[0]):int(0.97*imgs.shape[0])]:
        segmented = segment_lungs(img)
        if len(segmented) == 0:
            continue
        new_imgs[count] = (segmented[0] - np.mean(segmented[0])) / np.std(segmented[0])
        count += 1

    return new_imgs[:count]


def get_filtered_nodules(imgs, unet):
    nodules = []
    masks = unet.predict(imgs[:, np.newaxis, :, :], batch_size=4).astype(int)
    for i in range(masks.shape[0] - 1):
        mask = masks[i, 0]
        next_mask = masks[i + 1, 0]
        blobs = map(lambda x: np.array(x.centroid), measure.regionprops(measure.label(mask)))
        next_blobs = map(lambda x: np.array(x.centroid), measure.regionprops(measure.label(next_mask)))
        for blob in blobs:
            if not false_positive(next_blobs, blob, 15):
                r_min, r_max, c_min, c_max = get_patch_coord(blob, 50)
                nodules.append(imgs[i, r_min:r_max, c_min:c_max])

    return np.array(nodules, dtype=np.float32)


def get_masks(imgs, unet):
    masks = unet.predict(imgs[:, :, :, np.newaxis], batch_size=4).astype(int)
    return masks


def get_all_nodules(imgs, unet):
    nodules = []
    masks = unet.predict(imgs[:, np.newaxis, :, :], batch_size=4).astype(int)
    for idx, mask in enumerate(masks):
        blobs = map(lambda x: np.array(x.centroid), measure.regionprops(measure.label(mask[0])))
        for blob in blobs:
            r_min, r_max, c_min, c_max = get_patch_coord(blob, 50)
            nodules.append(imgs[idx, r_min:r_max, c_min:c_max])

    return np.array(nodules, dtype=np.float32)


def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)

    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)


def dice_coef_np(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)

    return (2. * intersection + 1) / (np.sum(y_true_f) + np.sum(y_pred_f) + 1)


def dice_coef_loss(y_true, y_pred):

    return -1*dice_coef(y_true, y_pred)


def crop_nodule(img, bbox):
    padding = 5
    y_start = np.clip(bbox[0][1] - padding, 0, 512)
    y_end = np.clip(bbox[1][1] + padding, 0, 512)
    x_start = np.clip(bbox[0][0] - padding, 0, 512)
    x_end = np.clip(bbox[1][0] + padding, 0, 512)
    cropped = img[y_start:y_end, x_start:x_end]
    cropped = cv2.resize(cropped, (50, 50))
    return cropped


def draw_labeled_bboxes(img, labels):
    copied = np.copy(img)
    bboxes = []
    # Iterate through all detected nodules
    for nodule_number in range(1, labels[1]+1):
        # Find pixels with each nodule_number label value
        nonzero = (labels[0] == nodule_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        width = np.max(nonzerox) - np.min(nonzerox)
        height = np.max(nonzerox) - np.max(nonzeroy)

        if width > 5 and height > 5:
            bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
            bboxes.append(bbox)
            # Draw the box on the image
            #cv2.rectangle(img, bbox[0], bbox[1], (10, 10, 10), 2)
            #copied = cv2.addWeighted(copied, 1.0, img, 1.0, 0.)

    # Return the image
    return copied, bboxes


def test_nodules():
    unet = load_model(lung_folder + '/unet19.h5', custom_objects={'dice_coef_loss': dice_coef_loss})

    df = pd.read_csv(lung_folder + 'kaggle/stage1_labels_subset.csv')
    test_df = pd.read_csv(lung_folder + 'kaggle/stage1_sample_submission_subset.csv')
    test_df = test_df[:2]

    tr_nodules = []

    for idx, patient in enumerate(test_df['id']):
        print(idx, patient)
        imgs = read_imgs(lung_folder + 'lung_subset_test/' + patient)
        tr_nodules.append((get_filtered_nodules(imgs, unet)), patient)

    np.save(lung_folder + 'test_nodules.npy', np.array(tr_nodules))


def train_masks():
    unet = load_model(lung_folder + '/unet19.h5', custom_objects={'dice_coef_loss': dice_coef_loss})
    train_df = pd.read_csv(lung_folder + 'kaggle/stage1_labels_subset.csv')
    num_samples = len(train_df)
    batch_size = 50
    batch = []
    print("Number of training samples:", num_samples)
    train_df.head()
    for i in range(28):
        nodules = []
        batch_start = i * batch_size
        batch_end = batch_start + batch_size
        ts = time.time()
        for idx, patient in train_df[batch_start:batch_end].iterrows():
            if idx % 5 == 0:
                print(i, idx, patient)
            slices = read_imgs(lung_folder + 'lung_subset/' + patient['id'])
            masks = get_masks(slices, unet)
            nodules.append((masks, patient['id'], patient['cancer']))
            del masks
            del slices
        np.save(lung_folder + 'train_masks%d.npy' % i, np.array(nodules))
        te = time.time()
        print("Batch runtime:", te - ts)


def crop_nodules_heatmap():
    unet = load_model(lung_folder + '/unet19.h5', custom_objects={'dice_coef_loss': dice_coef_loss})
    train_df = pd.read_csv(lung_folder + 'kaggle/stage1_labels.csv')
    num_samples = len(train_df)
    batch_size = 200
    batch = []
    print("Number of training samples:", num_samples)
    for i in range(7):
        nodules = []
        batch_start = i * batch_size
        batch_end = batch_start + batch_size
        ts = time.time()
        for idx, patient in train_df[batch_start:batch_end].iterrows():
            if idx % 50 == 0:
                print(i, idx, patient)
            patient_nodules = []
            # Get all slice masks from patient
            slices = read_imgs(lung_folder + 'lung_subset/' + patient['id'])

            # get predicted masks
            predicted = get_masks(slices, unet)

            # Create heatmap from all slices
            threshold = 2.0
            heatmap = np.sum(predicted, axis=0)[0]

            # threshold to keep hottest regions
            thresh_heatmap = np.copy(heatmap)
            thresh_heatmap[thresh_heatmap < threshold] = 0
            xy = thresh_heatmap.nonzero()
            thresh_heatmap[xy[0], xy[1]] = 1.

            # get bounding boxes on hottest nodule regions
            labels = label(thresh_heatmap)
            img_bbox, bboxes = draw_labeled_bboxes(np.copy(thresh_heatmap), labels)

            padding = 5
            # for each slice, keep only if dice coefficient > threshold
            for idx, predicted_slice in enumerate(predicted):
                for bbox in bboxes:
                    # isolate nodules
                    tmp = np.zeros((512, 512))
                    y_start = np.clip(bbox[0][1] - padding, 0, 512)
                    y_end = np.clip(bbox[1][1] + padding, 0, 512)
                    x_start = np.clip(bbox[0][0] - padding, 0, 512)
                    x_end = np.clip(bbox[1][0] + padding, 0, 512)
                    tmp[y_start:y_end, x_start:x_end] = 1

                    single_nodule_mask = np.logical_and(thresh_heatmap, tmp)

                    # Check if nodule covers area
                    dice_coefficient = dice_coef_np(single_nodule_mask, predicted_slice[0])
                    if dice_coefficient >= 0.40:
                        cropped_nodule = crop_nodule(slices[idx], bbox)
                        patient_nodules.append(cropped_nodule)

            nodules.append((np.array(patient_nodules), patient['id'], patient['cancer']))
            #print("Number of nodules detected for this patient",len(patient_nodules))

        np.save(lung_folder + 'cropped_heatmap_nodules_heat2_dice40%d.npy' % i, np.array(nodules))
        te = time.time()
        print("Batch runtime:", te - ts)


def crop_test_nodules():
    unet = load_model(lung_folder + '/unet19.h5', custom_objects={'dice_coef_loss': dice_coef_loss})
    test_df = pd.read_csv(lung_folder + 'kaggle/stage1_sample_submission_subset.csv')
    test_df = test_df[:2]
    num_samples = len(test_df)
    print("Number of testing samples:", num_samples)
    ts = time.time()
    nodules = []
    for idx, patient in test_df.iterrows():
        if idx % 25 == 0:
            print(idx, patient, len(nodules))
        patient_nodules = []
        # Get all slice masks from patient
        slices = read_imgs(patient['id'])

        # get predicted masks
        predicted = get_masks(slices, unet)

        # Create heatmap from all slices
        threshold = 2.0
        heatmap = np.sum(predicted, axis=0)[0]

        # threshold to keep hottest regions
        thresh_heatmap = np.copy(heatmap)
        thresh_heatmap[thresh_heatmap < threshold] = 0
        xy = thresh_heatmap.nonzero()
        thresh_heatmap[xy[0], xy[1]] = 1.

        # get bounding boxes on hottest nodule regions
        labels = label(thresh_heatmap)
        img_bbox, bboxes = draw_labeled_bboxes(np.copy(thresh_heatmap), labels)

        padding = 5
        # for each slice, keep only if dice coefficient > threshold
        for idx, predicted_slice in enumerate(predicted):
            for bbox in bboxes:
                # isolate nodules
                tmp = np.zeros((512, 512))
                y_start = np.clip(bbox[0][1] - padding, 0, 512)
                y_end = np.clip(bbox[1][1] + padding, 0, 512)
                x_start = np.clip(bbox[0][0] - padding, 0, 512)
                x_end = np.clip(bbox[1][0] + padding, 0, 512)
                tmp[y_start:y_end, x_start:x_end] = 1

                single_nodule_mask = np.logical_and(thresh_heatmap, tmp)

                # Check if nodule covers area
                dice_coefficient = dice_coef_np(single_nodule_mask, predicted_slice[0])
                if dice_coefficient >= 0.40:
                    cropped_nodule = crop_nodule(slices[idx], bbox)
                    patient_nodules.append(cropped_nodule)

        nodules.append((np.array(patient_nodules), patient['id']))
        #print("Number of nodules detected for this patient",len(patient_nodules))

    np.save(lung_folder + 'test_cropped_heatmap_nodules_heat2_dice40.npy', np.array(nodules))
    te = time.time()
    print("Batch runtime:", te - ts)


if __name__ == "__main__":
    crop_test_nodules()


Number of testing samples: 2
0 id        026470d51482c93efc18b9803159c960
cancer                                 0.5
Name: 0, dtype: object 0
Batch runtime: 675.7902569770813


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:324: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
nodule_files = glob.glob(lung_folder + '*cropped_heatmap_nodules*')
nodule_files
# nodule_files.sort()
data = np.load(nodule_files[0], allow_pickle=True)
data
# for nodule_file in nodule_files[0:]:
#     tmp = np.load(nodule_file, allow_pickle=True)
#     print(tmp)
#     data = np.vstack((data, tmp))
#     print(data)
  
# print("Number of patients:", data.shape)
# print("Patient shape:", data[0][0].shape)
# return data

array([[array([], dtype=float64), '026470d51482c93efc18b9803159c960'],
       [array([], dtype=float64), '031b7ec4fe96a3b035a8196264a8c8c3']],
      dtype=object)

In [ ]:
# classify nodules

import os
import pandas as pd
import numpy as np
import os.path as path
import glob
import pickle

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K


# kaggle_datafolder = '/media/data/kaggle/'
# kaggle_metadata = './data/kaggle/'
K.set_image_data_format('channels_last')


# def load_pickles():
#     nodule_files = glob.glob(kaggle_datafolder + '*.pickle')
#     with open(nodule_files[0], "rb") as f:
#         data = pickle.load(f, encoding='latin1')

#     for nodule_file in nodule_files[1:]:
#         with open(nodule_file, "rb") as f:
#             tmp = pickle.load(f, encoding='latin1')
#             data = np.hstack((data, tmp))
            
#     data = np.swapaxes(data, 0, 1)    
#     print("Number of patients:", data.shape)
#     print("Patient shape:", data[0][0].shape)
#     return data


def load_cropped_nodules():
    nodule_files = glob.glob(lung_folder + '*cropped_heatmap_nodules*')
    nodule_files.sort()
    data = np.load(nodule_files[0])
    for nodule_file in nodule_files[1:]:
        tmp = np.load(nodule_file)
        data = np.vstack((data, tmp))
	    
    print("Number of patients:", data.shape)
    print("Patient shape:", data[0][0].shape)
    return data


'''
Trains a classifier to classify nodes as cancerous/non-cancerous
'''
def train_cancer_classifier():
    data = load_cropped_nodules()

    # Prepare data for training
    nodules = data[0][0]
    labels = np.ones(data[0][0].shape[0]) * data[0][2]
    for idx, patient in enumerate(data[1:]):
        if patient[0].any():
            labels = np.concatenate((labels, np.ones(patient[0].shape[0]) * patient[2]))
            nodules = np.vstack((nodules, patient[0]))
	    
    print('Nodules shape:', nodules.shape)
    print('Labels shape:', labels.shape)

    num_classes = 2
    num_samples = nodules.shape[0]
    img_rows = nodules.shape[1]
    img_cols = nodules.shape[2]

    X_train, X_test, y_train, y_test = train_test_split(nodules, labels, test_size=0.33, random_state=42)

    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test = np_utils.to_categorical(y_test, num_classes)        
    print('Num train samples:', len(y_train))
    print('Num test samples:', len(y_test))
    
    epochs = 50
    batch_size = 128
    model = get_conv2d_model()
    print(X_train.shape)
    print(y_train.shape)
    model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=epochs,
                      verbose=2, validation_data=(X_test, y_test))
    score = model.evaluate(X_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    # save model
    model.save_weights(lung_folder + 'convnet_nodules.h5')    

def get_conv2d_model():
    model = Sequential()
    num_classes = 2
    input_shape = (50, 50, 1)
    model.add(Conv2D(32, 3, 3,
		     activation='relu',
		     input_shape=input_shape))

    model.add(Conv2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    return model


def classify_test_nodules():
    img_rows, img_cols = (50, 50)
    test_df = pd.read_csv('/content/drive/My Drive/lung/kaggle/stage1_sample_submission.csv')
    test_nodules = np.load(lung_folder + 'test_cropped_heatmap_nodules_heat2_dice40.npy', allow_pickle=True)
    model = get_conv2d_model()
    model.load_weights(lung_folder + 'convnet_nodules.h5')
        
    for idx, patient in test_df[:1].iterrows():
        test_nodules[idx][0] = test_nodules[idx][0].reshape((-1, img_rows, img_cols, 1))

        if test_nodules[idx][0].shape[0] != 0:
            prob = np.mean(model.predict(test_nodules[idx][0]), axis=0)[1]
            test_df.loc[idx, 'cancer'] = prob
        else:
            test_df.loc[idx, 'cancer'] = 0.5        

    test_df.head()
    test_df.to_csv('./submission.csv')


if __name__ == "__main__":
    #train_cancer_classifier()
    classify_test_nodules()


OSError: ignored

In [ ]:
# resnet50
# -*- coding: utf-8 -*-
'''ResNet50 model for Keras.

# Reference:

- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)

Adapted from code contributed by BigMoyan.
'''
from __future__ import print_function

import numpy as np
import warnings

from keras.layers import merge, Input
from keras.layers import Dense, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.preprocessing import image
import keras.backend as K
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
# from imagenet_utils import decode_predictions, preprocess_input


TH_WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_th_dim_ordering_th_kernels.h5'
TF_WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels.h5'
TH_WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_th_dim_ordering_th_kernels_notop.h5'
TF_WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'


def identity_block(input_tensor, kernel_size, filters, stage, block):
    '''The identity_block is the block that has no conv layer at shortcut

    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    '''
    nb_filter1, nb_filter2, nb_filter3 = filters
    if K.set_image_data_format('channels_first'):
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Convolution2D(nb_filter1, 1, 1, name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Convolution2D(nb_filter2, kernel_size, kernel_size,
                      border_mode='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Convolution2D(nb_filter3, 1, 1, name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = merge([x, input_tensor], mode='sum')
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    '''conv_block is the block that has a conv layer at shortcut

    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names

    Note that from stage 3, the first conv layer at main path is with subsample=(2,2)
    And the shortcut should have subsample=(2,2) as well
    '''
    nb_filter1, nb_filter2, nb_filter3 = filters
    if K.image_dim_ordering() == 'tf':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Convolution2D(nb_filter1, 1, 1, subsample=strides,
                      name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Convolution2D(nb_filter2, kernel_size, kernel_size, border_mode='same',
                      name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Convolution2D(nb_filter3, 1, 1, name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Convolution2D(nb_filter3, 1, 1, subsample=strides,
                             name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = merge([x, shortcut], mode='sum')
    x = Activation('relu')(x)
    return x


def ResNet50(include_top=True, weights='imagenet',
             input_tensor=None):
    '''Instantiate the ResNet50 architecture,
    optionally loading weights pre-trained
    on ImageNet. Note that when using TensorFlow,
    for best performance you should set
    `image_dim_ordering="tf"` in your Keras config
    at ~/.keras/keras.json.

    The model and the weights are compatible with both
    TensorFlow and Theano. The dimension ordering
    convention used by the model is the one
    specified in your Keras config file.

    # Arguments
        include_top: whether to include the 3 fully-connected
            layers at the top of the network.
        weights: one of `None` (random initialization)
            or "imagenet" (pre-training on ImageNet).
        input_tensor: optional Keras tensor (i.e. xput of `layers.Input()`)
            to use as image input for the model.

    # Returns
        A Keras model instance.
    '''
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')
    # Determine proper input shape
    if K.set_image_data_format('channels_first'):
        if include_top:
            input_shape = (3, 224, 224)
        else:
            input_shape = (3, None, None)
    else:
        if include_top:
            input_shape = (224, 224, 3)
        else:
            input_shape = (None, None, 3)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor)
        else:
            img_input = input_tensor
    if K.set_image_data_format('channels_first'):
        bn_axis = 3
    else:
        bn_axis = 1

    x = ZeroPadding2D((3, 3))(img_input)
    x = Convolution2D(64, 7, 7, subsample=(2, 2), name='conv1')(x)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    x = AveragePooling2D((7, 7), name='avg_pool')(x)

    if include_top:
        x = Flatten()(x)
        x = Dense(1000, activation='softmax', name='fc1000')(x)

    model = Model(img_input, x)

    # load weights
    if weights == 'imagenet':
        print('K.image_dim_ordering:', K.image_dim_ordering())
        if K.set_image_data_format('channels_first'):
            if include_top:
                weights_path = get_file('resnet50_weights_th_dim_ordering_th_kernels.h5',
                                        TH_WEIGHTS_PATH,
                                        cache_subdir='models',
                                        md5_hash='1c1f8f5b0c8ee28fe9d950625a230e1c')
            else:
                weights_path = get_file('resnet50_weights_th_dim_ordering_th_kernels_notop.h5',
                                        TH_WEIGHTS_PATH_NO_TOP,
                                        cache_subdir='models',
                                        md5_hash='f64f049c92468c9affcd44b0976cdafe')
            model.load_weights(weights_path)
            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image dimension ordering convention '
                              '(`image_dim_ordering="th"`). '
                              'For best performance, set '
                              '`image_dim_ordering="tf"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')
                convert_all_kernels_in_model(model)
        else:
            if include_top:
                weights_path = get_file('resnet50_weights_tf_dim_ordering_tf_kernels.h5',
                                        TF_WEIGHTS_PATH,
                                        cache_subdir='models',
                                        md5_hash='a7b3fe01876f51b976af0dea6bc144eb')
            else:
                weights_path = get_file('resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                        TF_WEIGHTS_PATH_NO_TOP,
                                        cache_subdir='models',
                                        md5_hash='a268eb855778b3df3c7506639542a6af')
            model.load_weights(weights_path)
            if K.backend() == 'theano':
                convert_all_kernels_in_model(model)
    return model


if __name__ == '__main__':
    model = ResNet50(include_top=True, weights='imagenet')

    img_path = 'elephant.jpg'
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    print('Input image shape:', x.shape)

    preds = model.predict(x)
    print('Predicted:', decode_predictions(preds))


TypeError: ignored